In [3]:
from IPython.display import display, HTML
import pandas as pd
import time
import re
import os
import sys
import ipywidgets as widgets
from bokeh.io import output_file, show
from bokeh.models import CheckboxGroup
from ipywidgets import interact, interactive, fixed, Layout, HBox, VBox

if './modules/' not in sys.path:
    sys.path.append('./modules')
from drop_downs import *
from build_statements import *
from query import *
from plot import *   
from outliers import *
from tasks import *

default = ('ignore','   ')
def query_plot(b):
    with output:
        display(HTML('<h2>Results</h2>'))

        features_values={'prescribing':[bnf_code_dd.value,'bnf_code',prescribing_measures_dd.value,\
                                        month_dd.value,year_dd.value],\
                'prevalence':[prevalence_indications_dd.value,'indication',prevalence_measures_dd.value,'',year_dd.value],\
                'gender':[gender_dd.value,'sex',gender_age_measures_dd.value,\
                          month_dd.value,year_dd.value],\
                'age_groups':[age_groups_dd.value,'age_group',gender_age_measures_dd.value,\
                              month_dd.value,year_dd.value],\
                'deprivation':[deprivation_dd.value,'deprivation_index',deprivation_measures_dd.value,'',year_dd.value]}
        
        headers=['value','value_header','measure_header','month','year']
        fvdf=pd.DataFrame.from_dict(features_values, orient='index',columns=headers)
        fvdf=fvdf[fvdf['value']!=default]
        fvdf=fvdf.fillna('not_selected')
        features = fvdf.index
        fvdf['table'] = get_tables(fvdf,features)
        #print (fvdf) ##print (fvdf) #pandas 1: #print (fvdf.to_markdown())
        
        # Query format
        title=get_title(fvdf)
        #print ('Query:\n',title)
        
        # check/#print errors: (query_format(fvdf))
        
        # Build query statements for the features selected
        #print ('\n**** statements ****\n')
        statements = {}
        
        # If both age and gender were selected
        if ({'age_groups','gender'}) <= set(features):
            #print ( 'age groups:', fvdf.loc ['age_groups','value'],fvdf.loc ['age_groups','value_header'])
            #print ( 'gender:', fvdf.loc ['gender','value'],fvdf.loc ['gender','value_header'])
            built_statement = combined_statement(fvdf).build()
            #print (built_statement)
            statements['age_groups_gender'] = built_statement
            features = [feature for feature in features if feature not in ['age_groups','gender']]
        
        #print ('features remaining:',features)
        for feature in features:
            ##print ('feature:', feature)
            st = statement (feature,fvdf)
            #print (st.value,st.measure_header,st.table)
            built_statement = '{sel}\n{whe}\n{grp}'.format (sel=st.select (feature),\
            whe=st.where(feature),grp=st.groupby(feature))
            # built_statement = st.build(feature)
            ##print ('statement:\n',built_statement)
            ##print ('*****')
            statements[feature] = built_statement

        # Locations data
        location_statement = "SELECT practice,practice_code,ccg,region,sub_region,longitudemerc,latitudemerc\
                     FROM practices_locations"
        statements['location']=location_statement
        
        #print ('\n\nstatements\n\n')
        #for feature, built_statement in statements.items():
        #print ('{f}:\n {bs}\n*****'.format(f=feature, bs=built_statement))
        
        # Use the statement to query the relevant tables
        queries_df=query(statements)
        
        #print ("****** Queries Results *******\n")
        #print (queries_df.columns)
        #print (queries_df.head())
        #print (queries_df.info())
        
        # Outliers in value_columns
        geo_columns=['practice_code', 'practice', 'ccg','region','sub_region', 'longitudemerc','latitudemerc']
        value_headers=[h for h in queries_df.columns if h not in geo_columns]
        #print ('value_headers:',value_headers)
        outliers_indices_collect=[]
        
        #print ("\n\n****** Ouliers Detection and Removal *******\n")

        for header in value_headers:
            outliers_indices = detect_indices(queries_df,header)
            #print (len (outliers_indices))
            #print (outliers_indices)
            outliers_indices_collect+=outliers_indices
        #print (len(outliers_indices_collect))
        #print (outliers_indices_collect)
        queries_df=queries_df[~queries_df.index.isin(outliers_indices_collect)]
        
        #print ("\n\n****** Results, Outliers removed *******\n")
        #print (queries_df.columns)
        #print (queries_df.head())
        #print (queries_df.info())
        
        #print (value_headers)
        queries_df.to_csv('queries_df.csv',index=False)
        #print ('\n\nplot')
        
        map_plot(queries_df,title)
        
        
# Dashboard
output = widgets.Output()
button.on_click(query_plot)
display(HTML('<h1>Geo Health Dashboard</h1>'))
VBox(children=[tabs,button_date_box,output])

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')